In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import rads_extraction
import tec_interpolation
import datetime as dt


- RADS model 
- IMIC model 

In [2]:
def time_convert(date:str):
    '''
    Function that converts a date string from "HH:MM DD/MM/YYYY" to "YYYY-MM-DDTHH:MM:SS"
    '''
    date = date.split(' ')
    time = date[0].split(':')
    date = date[1].split('/')
    return f'{date[2]}-{date[1]}-{date[0]}T{time[0]}:{time[1]}:00'

In [3]:
def imic(alpha, beta, f=13.575e9, filename=None):
    '''docstring TODO'''
    tecu = 1e16
        
    time, lat, lon, sla_uncorrected = rads_extraction.extract_rads(filename) #TODO check filenames
    tec_GPS = tec_interpolation.mass_interpolate(lon, lat, time)
    
    if isinstance(alpha, float):
        return time, lat, lon, (40.3/f**2)*alpha*beta*(tec_GPS*tecu) + sla_uncorrected
    else:
        sla = [(40.3/f**2)*alpha[i]*beta[i]*(tec_GPS*tecu) + sla_uncorrected for i in range(len(alpha))]
        return time, lat, lon, tuple(sla)

## Validation with Sentinel-3A

In [4]:
alpha, beta = 0.8452, 0.936

time_1, lat, lon, sla_uncorrected = rads_extraction.extract_rads(r'final_data\\s3a_2016_1500_noiono.nc') #TODO check filenames
time_2, lat, lon, sla_true        = rads_extraction.extract_rads(r'final_data\\s3a_2016_1500.nc')        #TODO check filenames
time_3, lat, lon, sla_RADS_gim    = rads_extraction.extract_rads(r'final_data\\s3a_2016_1500_noiono.nc') #TODO check filenames
time_4, lat, lon, (sla_IMIC_gim, sla_unscaled) = imic(alpha=(alpha, 1), beta=(beta, 1), filename=r'final_data\\s3a_2016_1500_noiono.nc')

time_lst = [time_1, time_2, time_3, time_4]


check = [all(x == y for y in time_lst) for x in time_lst]

C:\Users\Alfon\OneDrive - Delft University of Technology\02 Second Year\BSc2 Q3\AE2224-I - Test, Analysis & Simulation\02 Code\TestAnalysisSimulation\main\RADS\final_data\\s3a_2016_1500_noiono.nc
C:\Users\Alfon\OneDrive - Delft University of Technology\02 Second Year\BSc2 Q3\AE2224-I - Test, Analysis & Simulation\02 Code\TestAnalysisSimulation\main\RADS\final_data\\s3a_2016_1500.nc
C:\Users\Alfon\OneDrive - Delft University of Technology\02 Second Year\BSc2 Q3\AE2224-I - Test, Analysis & Simulation\02 Code\TestAnalysisSimulation\main\RADS\final_data\\s3a_2016_1500_noiono.nc
C:\Users\Alfon\OneDrive - Delft University of Technology\02 Second Year\BSc2 Q3\AE2224-I - Test, Analysis & Simulation\02 Code\TestAnalysisSimulation\main\RADS\final_data\\s3a_2016_1500_noiono.nc
Checking availability of source GIMs...
Downloaded and decompressed: main/temp/jpld1260.16i.nc.gz
Downloaded and decompressed: main/temp/jpld1270.16i.nc.gz
Downloaded and decompressed: main/temp/jpld1280.16i.nc.gz
Downloade

In [ ]:

diff_unscaled = sla_unscaled - sla_true
diff_IMIC     = sla_IMIC_gim - sla_true
diff_RADS_gim = sla_unscaled - sla_true

diff_arr = np.array([diff_unscaled, diff_IMIC, diff_RADS_gim])

unit_change = 100

mean_arr = np.mean(unit_change*diff_arr, axis=-1)
std_arr  = np.std( unit_change*diff_arr, axis=-1)
# printing the results
print(f'       | Unscaled | IMIC | RADS GIM')
print(f'Mean   | {mean_arr}')
print(f'STD    | {std_arr}')

# ----------- plotting ----------------

# setting the x-axis appropriately
dates = [dt.datetime.fromisoformat(time_convert(date)) for date in time_1]
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y/%m/%d'))
plt.gca().xaxis.set_major_locator(mdates.DayLocator(interval=50))
plt.gcf().autofmt_xdate()

# 
plt.plot(dates, unit_change*diff_unscaled, color='orange', label=r'unscaled IMIC model ($\alpha=\beta=1$)')
plt.plot(dates, unit_change*diff_IMIC    , color='green' , label='scaling with IMIC model')
plt.plot(dates, unit_change*diff_RADS_gim, color='blue'  , label='scaling with RADS GIM model')

plt.grid(True, alpha=0.6)
plt.legend()
plt.xlabel('Time')
plt.ylabel('diff wrt. dual-frequency measurement [cm]')



In [26]:
import numpy as np

def imic(alpha, beta, f=13.575e9, filename=None):
    '''docstring TODO'''
    tecu = 1e16
        
    time, lat, lon, sla_uncorrected = np.zeros((4, 16))
    tec_GPS = np.ones((16))

    if isinstance(alpha, float) or isinstance(alpha, int):
        return time, lat, lon, (40.3/f**2)*alpha*beta*(tec_GPS*tecu) + sla_uncorrected
    else:
        sla = [((40.3/f**2)*alpha[i]*beta[i]*(tec_GPS*tecu) + sla_uncorrected) for i in range(len(alpha))]
        return time, lat, lon, tuple(sla)

In [27]:
alpha, beta = 0.8452, 0.936
imic(alpha=(alpha, 2), beta=(beta, 0.7))

(array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 (array([0.00173006, 0.00173006, 0.00173006, 0.00173006, 0.00173006,
         0.00173006, 0.00173006, 0.00173006, 0.00173006, 0.00173006,
         0.00173006, 0.00173006, 0.00173006, 0.00173006, 0.00173006,
         0.00173006]),
  array([0.00306163, 0.00306163, 0.00306163, 0.00306163, 0.00306163,
         0.00306163, 0.00306163, 0.00306163, 0.00306163, 0.00306163,
         0.00306163, 0.00306163, 0.00306163, 0.00306163, 0.00306163,
         0.00306163])))

In [18]:
import numpy as np

x1 = np.array([0, 0, 0, 0])
x2 = np.array([0, 0, 1, 0])
x3 = np.array([0, 0, 0, 0])

x_lst = np.array([x1, x2, x3])

np.array([np.isclose(x_lst, xi).all() for xi in x_lst]).all()

False

In [19]:
import datetime as dt

In [23]:
f'{dt.datetime.now():%Y-%m-%d %H:%M}'

'2024-05-12 13:23'

In [25]:
from directory_paths import res_dir

In [27]:
import pandas as pd
import os
alpha, beta = 0.3423, 0.4532
x1 = np.array([0, 2.5468, 0])
x2 = np.array([8.1681, 0, 1])
df = pd.DataFrame([x1, x2], ['Mean (cm)', 'Std (cm)'], ['Unscaled', 'IMIC', 'RADS GIM'])

res_fname = f'{dt.datetime.now():%Y-%m-%d %H.%M} - TEST - a={alpha} b={beta}.txt'

with open(os.path.join(res_dir, res_fname), 'a') as f:
    df.to_string(buf=f)


In [28]:
os.path.join('test', '.png')

'test\\.png'

In [29]:
(0.8737+0.795)/2

0.83435

In [2]:
time_1, lat1, lon1, sla_uncorrected = rads_extraction.extract_rads(r'final_data\\s3a_2016_1500_noiono.nc') #TODO check filenames
time_2, lat, lon, sla_true        = rads_extraction.extract_rads(r'final_data\\s3a_2016_1500.nc')        #TODO check filenames

len(time_1), len(time_2)

C:\Users\Alfon\OneDrive - Delft University of Technology\02 Second Year\BSc2 Q3\AE2224-I - Test, Analysis & Simulation\02 Code\TestAnalysisSimulation\main\RADS\final_data\\s3a_2016_1500_noiono.nc
C:\Users\Alfon\OneDrive - Delft University of Technology\02 Second Year\BSc2 Q3\AE2224-I - Test, Analysis & Simulation\02 Code\TestAnalysisSimulation\main\RADS\final_data\\s3a_2016_1500.nc


(6825, 6812)

In [3]:
time_1, lat1, lon1, sla_uncorrected = rads_extraction.extract_rads(r'final_data\\s3a_2016_1500_noiono_v2.nc') #TODO check filenames
time_2, lat, lon, sla_true        = rads_extraction.extract_rads(r'final_data\\s3a_2016_1500_v2.nc')        #TODO check filenames

len(time_1), len(time_2)

C:\Users\Alfon\OneDrive - Delft University of Technology\02 Second Year\BSc2 Q3\AE2224-I - Test, Analysis & Simulation\02 Code\TestAnalysisSimulation\main\RADS\final_data\\s3a_2016_1500_noiono_v2.nc
C:\Users\Alfon\OneDrive - Delft University of Technology\02 Second Year\BSc2 Q3\AE2224-I - Test, Analysis & Simulation\02 Code\TestAnalysisSimulation\main\RADS\final_data\\s3a_2016_1500_v2.nc


(6825, 6812)